# 2. Feature extraction and fusion for unimodal classification

<a id='task2'></a>
<div class=" alert alert-warning">
    <b>Assigment.</b> <b>Task 2.</b>

Use the training dataset prepared in task 1. to build models based on the combination of principal component analysis (PCA), linear discriminant analysis (LDA), and nearest neighbour (NN) classifier for each modality separately and evaluate the model on test dataset. Do the subtasks given as
<br>
<br>
<p> <b>2.1</b> Calculate PCA and LDA transformations to reduce the dimensionality of accelerometer data (e.g., using scikit-learn implementations). Before transformations downsample data from 100 Hz to 25 Hz (using scipy.signal.resample) to get 125x3 matrix of data for each 5 sec window. You should also standardize the values to zero mean and unit variance before the transformations. Using training dataset, fit PCA with 5-dimensional subspace (i.e., choosing the 5 largest principal components) and fit LDA with 5-dimensional subspace. Transform both train and test examples to this low-dimensional feature representation. Concatenate each sequence to single vector size of 3x(5+5). Perform the fusion of PCA and LDA similar manner as presented in Lecture 3 (pages 24-25) using NN method. Evaluate the performance on testset. Show confusion matrix and F1 scores of the results. </p>
<br>
<p> <b>2.2</b> Use PCA and LDA transformations to reduce the dimensionality of depth images. You should also standardize the values to zero mean and unit variance before the transformations. Fit PCA and LDA for all training images (12x16, 192-dimensional in vectorized form) by choosing 5-dimensional subspace for both PCA and LDA. Transform both train and test examples to this low-dimensional feature representation. Concatenate each sequence to single vector size of 5x1x(5+5). Similar to task 2.1, do the PCA and LDA fusion using NN and evaluate the performance on testset. Show confusion matrix and F1 scores of the results. </p>
<br>
Document your work, evaluate the results, and analyse the outcomes in each subtasks 2.1-2.2.

</div>

## 2.1 Calculate PCA and LDA transformations to reduce the dimensionality of accelerometer data

In [ ]:
# 2.1 Imports:
import importlib, utilities.fun_two, utilities.fun_one
importlib.reload(utilities.fun_two)
importlib.reload(utilities.fun_one)
from utilities.fun_two import acccelerometer_resample, PcaActApplier, LdaActApplier, act_fusion, Standardizer
from utilities.fun_one import visualize
import time
import pandas as pd

In [ ]:
%store -r training_records
training_records = training_records
%store -r testing_records
testing_records = testing_records

In [ ]:
### Your code begins here ###
# Resample data:
resample_samples = 125
act_train = pd.DataFrame()
act_train['df'] = acccelerometer_resample(training_records,resample_samples)
train_labels = training_records[training_records.sensor_code=='act'].exercise_id.apply(lambda x: int(x))
act_test = pd.DataFrame()
act_test['df'] = acccelerometer_resample(testing_records,resample_samples)
test_labels = testing_records[testing_records.sensor_code=='act'].exercise_id.apply(lambda x: int(x))

In [ ]:
start_time = time.time() # Measure execution time

#Standardize the Data:
act_s = Standardizer()
act_s.fit(act_train)
act_train['df'] = act_s.transform(act_train)
act_test['df'] = act_s.transform(act_test)

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time() # Measure execution time

#PCA
n_components = 5
pca = PcaActApplier(n_components)
pca.fit(act_train['df'])
act_pca_train = pca.transform(act_train['df'])
act_pca_test = pca.transform(act_test['df'])

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time() # Measure execution time

#LDA:
n_components = 5
lda = LdaActApplier(n_components)
lda.fit(act_train['df'],train_labels)
act_lda_train = lda.transform(act_train['df'])
act_lda_test = lda.transform(act_test['df'])

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time() # Measure execution time

# Fusion:
train_pred_labels = act_fusion(act_pca_train,act_lda_train,act_pca_train,act_lda_train,train_labels)
test_pred_labels = act_fusion(act_pca_train,act_lda_train,act_pca_test,act_lda_test,train_labels)

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
# Visualisation:
visualize(train_pred_labels,train_labels,test_pred_labels,test_labels,main_title="Accelerometer sensor used to classify the exercises")
### Your code ends here ###

## 2.2 Use PCA and LDA transformations to reduce the dimensionality of depth images

In [ ]:
# 2.2 Imports:
import importlib, utilities.fun_two, utilities.fun_one
importlib.reload(utilities.fun_two)
importlib.reload(utilities.fun_one)
from utilities.fun_two import concat_pca_lda, PcaDcApplier, LdaDcApplier, classifyNN, Standardizer
from utilities.fun_one import visualize

In [ ]:
### Your code begins here ###
start_time = time.time() # Measure execution time

'''Work only with rows with dc'''
dc_train_records = training_records[training_records['sensor'] == 'dc']
dc_test_records = testing_records[testing_records['sensor'] == 'dc']

reduced_dimensions = 5

end_time = time.time()
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time() # Measure execution time

'''Standardize the dc data'''
standardizer = Standardizer()
standardizer.fit(dc_train_records)
standardized_dc_train_records = standardizer.transform(dc_train_records)
standardized_dc_test_records = standardizer.transform(dc_test_records)

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time() # Measure execution time

'''Fit PCA for depth sensor on standardized train dataset and transform test an train dataset'''
pca_applier = PcaDcApplier(reduced_dimensions)
pca_applier.fit(standardized_dc_train_records)

pca_dc_train_records = pca_applier.transform(standardized_dc_train_records)
pca_dc_test_records = pca_applier.transform(standardized_dc_test_records)

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time() # Measure execution time

'''Fit PCA for depth sensor on standardized train dataset and transform test an train dataset'''
lda_applier = LdaDcApplier(reduced_dimensions)
lda_applier.fit(standardized_dc_train_records)

lda_dc_train_records = lda_applier.transform(standardized_dc_train_records)
lda_dc_test_records = lda_applier.transform(standardized_dc_test_records)

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time() # Measure execution time
'''Concatenate PCA and LDA features of depth sensor data'''
concat_dc_train_records = concat_pca_lda(pca_dc_train_records, lda_dc_train_records)
concat_dc_test_records = concat_pca_lda(pca_dc_test_records, lda_dc_test_records)

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time() # Measure execution time

'''Get estimated labels for train dataset calculated by Nearest Neighbour algorithm on depth sensor data'''
est_train_labels = classifyNN(train_data = concat_dc_train_records, test_data=concat_dc_train_records)

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
start_time = time.time() # Measure execution time

'''Get estimated labels for test dataset calculated by Nearest Neighbour algorithm on depth sensor data'''
est_test_labels = classifyNN(train_data = concat_dc_train_records, test_data=concat_dc_test_records)

end_time = time.time() # Measure execution time
print("Execution Time: ", end_time - start_time)

In [ ]:
'''Visualize the results of NN classification for depth sensor data'''
visualize(est_train_labels['estimated_label'],est_train_labels['real_label'],est_test_labels['estimated_label'],est_test_labels['real_label'],main_title="Depth sensor used to classify the exercises")
### Your code ends here ###